# Gold Recovery Project

## Introduction

## Preparing Data

In [67]:
# imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score,  roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import plotly.express as px

All imports

In [68]:
data_full = pd.read_csv(r"C:\Users\alexi\Desktop\Coding Projects\Gold-Project\Gold-Project\gold_recovery_full.csv")
data_test = pd.read_csv(r"C:\Users\alexi\Desktop\Coding Projects\Gold-Project\Gold-Project\gold_recovery_test.csv")
data_train = pd.read_csv(r"C:\Users\alexi\Desktop\Coding Projects\Gold-Project\Gold-Project\gold_recovery_train.csv")

Imported all csvs

In [69]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                22716 non-null  object 
 1   final.output.concentrate_ag                         22627 non-null  float64
 2   final.output.concentrate_pb                         22629 non-null  float64
 3   final.output.concentrate_sol                        22331 non-null  float64
 4   final.output.concentrate_au                         22630 non-null  float64
 5   final.output.recovery                               20753 non-null  float64
 6   final.output.tail_ag                                22633 non-null  float64
 7   final.output.tail_pb                                22516 non-null  float64
 8   final.output.tail_sol                               22445 non-null  float64


In [70]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   date                                        5856 non-null   object 
 1   primary_cleaner.input.sulfate               5554 non-null   float64
 2   primary_cleaner.input.depressant            5572 non-null   float64
 3   primary_cleaner.input.feed_size             5856 non-null   float64
 4   primary_cleaner.input.xanthate              5690 non-null   float64
 5   primary_cleaner.state.floatbank8_a_air      5840 non-null   float64
 6   primary_cleaner.state.floatbank8_a_level    5840 non-null   float64
 7   primary_cleaner.state.floatbank8_b_air      5840 non-null   float64
 8   primary_cleaner.state.floatbank8_b_level    5840 non-null   float64
 9   primary_cleaner.state.floatbank8_c_air      5840 non-null   float64
 10  primary_clea

In [71]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                16860 non-null  object 
 1   final.output.concentrate_ag                         16788 non-null  float64
 2   final.output.concentrate_pb                         16788 non-null  float64
 3   final.output.concentrate_sol                        16490 non-null  float64
 4   final.output.concentrate_au                         16789 non-null  float64
 5   final.output.recovery                               15339 non-null  float64
 6   final.output.tail_ag                                16794 non-null  float64
 7   final.output.tail_pb                                16677 non-null  float64
 8   final.output.tail_sol                               16715 non-null  float64


all of these data sets seem very inconsistent in entries so im gonna look into this


In [72]:
missing_rows_df = data_full[data_full.isnull().any(axis=1)]
missing_rows_df

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
47,2016-01-16 23:00:00,4.591462,10.628846,5.688961,46.663335,55.088448,10.694584,1.671644,17.910312,2.686782,...,13.997488,-551.224653,11.971272,-550.095619,9.981163,-502.869109,7.941184,-500.623610,12.018837,-600.916665
48,2016-01-17 00:00:00,5.296830,10.597469,NaN,44.015642,41.962008,11.975654,1.846558,8.138408,2.889469,...,14.006476,-549.388727,11.975967,-550.778624,9.989726,-499.607365,7.958252,-498.882560,11.967977,-599.440603
49,2016-01-17 01:00:00,7.256114,9.017677,NaN,43.389252,NaN,12.100582,1.758360,4.606019,3.202483,...,14.032364,-549.326033,11.987133,-549.583570,10.094584,-492.041563,8.054770,-498.802262,11.961330,-597.272218
50,2016-01-17 02:00:00,8.162896,8.877528,NaN,40.715973,49.884856,10.381465,1.024030,12.974257,2.739224,...,13.984906,-550.563329,11.999730,-550.549773,9.941869,-503.282068,8.018850,-500.970716,11.991078,-598.475929
53,2016-01-17 05:00:00,5.919503,11.078044,6.215769,42.922611,71.308281,10.866006,1.577753,17.550397,2.476280,...,13.972224,-548.238797,12.042964,-550.430774,9.989624,-499.606754,7.950446,-500.352648,12.011728,-599.043399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22702,2018-08-17 21:59:59,3.599454,12.308172,6.512304,45.477834,59.871040,8.882691,3.036575,9.421421,1.826293,...,22.988173,-500.989518,20.005718,-500.154032,17.987716,-501.022243,13.020843,-500.218430,20.011456,-499.842173
22703,2018-08-17 22:59:59,3.540800,12.423927,6.702631,45.323979,70.477689,8.866079,2.816357,9.838458,1.439220,...,23.036427,-500.990672,20.023346,-500.147796,18.010037,-500.299851,12.982786,-500.097885,20.003557,-506.135132
22704,2018-08-17 23:59:59,3.573966,12.199649,6.584768,45.440494,74.594019,9.042359,2.980534,9.737694,1.455128,...,23.011485,-500.714430,20.058045,-499.066195,17.990312,-499.278862,13.002126,-499.724599,20.004410,-500.347623
22705,2018-08-18 00:59:59,3.525247,12.121578,6.605982,45.785582,75.185602,9.041052,3.208360,9.792629,1.534906,...,23.035611,-501.134671,19.984883,-501.051735,17.948892,-500.147451,13.009772,-500.081309,20.012084,-501.651764


there seems to be no correlation to be made for the missing values so i am going to fill them with the column median

In [73]:
def fill_missing_with_median(df):
    return df.fillna(df.median(numeric_only=True))

data_full = fill_missing_with_median(data_full)
data_test = fill_missing_with_median(data_test)
data_train = fill_missing_with_median(data_train)

In [74]:
data_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                22716 non-null  object 
 1   final.output.concentrate_ag                         22716 non-null  float64
 2   final.output.concentrate_pb                         22716 non-null  float64
 3   final.output.concentrate_sol                        22716 non-null  float64
 4   final.output.concentrate_au                         22716 non-null  float64
 5   final.output.recovery                               22716 non-null  float64
 6   final.output.tail_ag                                22716 non-null  float64
 7   final.output.tail_pb                                22716 non-null  float64
 8   final.output.tail_sol                               22716 non-null  float64


ive filled all missing values with median every column except date this is an ok response as we dont need this for future calculations

In [75]:
data_full.duplicated().sum()

np.int64(0)

In [76]:
data_test.duplicated().sum()

np.int64(0)

In [77]:
data_train.duplicated().sum()

np.int64(0)

no duplicated rows so we can work with the data from here

In [78]:
required_columns = [
    'final.output.concentrate_au',
    'rougher.input.feed_au',
    'final.output.tail_au'
]

sample_row = data_train.dropna(subset=required_columns).iloc[0]
C = sample_row['final.output.concentrate_au']
F = sample_row['rougher.input.feed_au']
T = sample_row['final.output.tail_au']

# Calculate recovery
recovery = (C * (F - T)) / (F * (C - T)) * 100
C, F, T, recovery

(np.float64(42.19202041625977),
 np.float64(6.486149787902832),
 np.float64(2.143149375915528),
 np.float64(70.54121591421571))

In [79]:
clean_data = data_train.dropna(subset=required_columns)
row_index = clean_data.index[0]
sample_row = clean_data.iloc[0]

print(f"Original index of selected row: {row_index}")
print(sample_row.to_dict())



Original index of selected row: 0
{'date': '2016-01-15 00:00:00', 'final.output.concentrate_ag': 6.055402755737305, 'final.output.concentrate_pb': 9.8896484375, 'final.output.concentrate_sol': 5.50732421875, 'final.output.concentrate_au': 42.19202041625977, 'final.output.recovery': 70.54121591421571, 'final.output.tail_ag': 10.411961555480955, 'final.output.tail_pb': 0.89544677734375, 'final.output.tail_sol': 16.904296875, 'final.output.tail_au': 2.143149375915528, 'primary_cleaner.input.sulfate': 127.09200286865234, 'primary_cleaner.input.depressant': 10.128294944763184, 'primary_cleaner.input.feed_size': 7.25, 'primary_cleaner.input.xanthate': 0.9887590408325196, 'primary_cleaner.output.concentrate_ag': 8.547551155090332, 'primary_cleaner.output.concentrate_pb': 10.3896484375, 'primary_cleaner.output.concentrate_sol': 19.529296875, 'primary_cleaner.output.concentrate_au': 34.174427032470696, 'primary_cleaner.output.tail_ag': 14.93652629852295, 'primary_cleaner.output.tail_pb': 2.5349

the column value itself is 87.10776292825425 but the recovery formula reads 70.54121591421571 i am gonna use MAE to calculate the difference

In [80]:
# Calculate MAE
actual = 87.10776292825425
predicted = 70.54121591421571

mae = abs(actual - predicted)
print(f"MAE: {mae:.4f}")

MAE: 16.5665


In [81]:
missing_in_test = sorted(set(data_train.columns) - set(data_test.columns))
missing_in_test

['final.output.concentrate_ag',
 'final.output.concentrate_au',
 'final.output.concentrate_pb',
 'final.output.concentrate_sol',
 'final.output.recovery',
 'final.output.tail_ag',
 'final.output.tail_au',
 'final.output.tail_pb',
 'final.output.tail_sol',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_au',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.tail_ag',
 'primary_cleaner.output.tail_au',
 'primary_cleaner.output.tail_pb',
 'primary_cleaner.output.tail_sol',
 'rougher.calculation.au_pb_ratio',
 'rougher.calculation.floatbank10_sulfate_to_au_feed',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'rougher.calculation.sulfate_to_au_concentrate',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_au',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_sol',
 'rougher.output.recovery',
 'rougher.output.tail_ag',
 'rougher.output.tail_au',
 'rougher.output.ta

these are the 33 features missing from the train set and they are all float types and all seem to be in relation to the recovery process

## Analyze the Data

In [82]:
# Define the concentration columns in the correct purification order
concentration_columns_ordered = {
    "Feed": ['rougher.input.feed_au', 'rougher.input.feed_ag', 'rougher.input.feed_pb'],
    "Rough Purification": ['rougher.output.concentrate_au', 'rougher.output.concentrate_ag', 'rougher.output.concentrate_pb'],
    "Primary Purification": ['primary_cleaner.output.concentrate_au', 'primary_cleaner.output.concentrate_ag', 'primary_cleaner.output.concentrate_pb'],
    "Secondary Purification": ['secondary_cleaner.output.tail_au', 'secondary_cleaner.output.tail_ag', 'secondary_cleaner.output.tail_pb'],  # intermediate tail
    "Final Purification": ['final.output.concentrate_au', 'final.output.concentrate_ag', 'final.output.concentrate_pb']
}

# Recalculate average concentrations for each stage
summary_updated = {}
for stage, cols in concentration_columns_ordered.items():
    mean_values = data_train[cols].mean()
    summary_updated[stage] = {
        'Gold (Au)': mean_values[0],
        'Silver (Ag)': mean_values[1],
        'Lead (Pb)': mean_values[2]
    }

# Create the updated summary DataFrame
concentration_summary_ordered = pd.DataFrame.from_dict(summary_updated, orient='index')
concentration_summary_ordered

C:\Users\alexi\AppData\Local\Temp\ipykernel_3348\2855755530.py:15: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\alexi\AppData\Local\Temp\ipykernel_3348\2855755530.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\alexi\AppData\Local\Temp\ipykernel_3348\2855755530.py:17: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\alexi\AppData\Local\Temp\ipykernel_3348\2855755530.py:15: FutureWarning:

Series.__getit

,Gold (Au),Silver (Ag),Lead (Pb)
Feed,7.172060,7.832862,3.219987
Rough Purification,17.412450,10.584752,6.883743
Primary Purification,29.181542,7.425434,8.641498
Secondary Purification,3.935446,13.023731,5.320013
Final Purification,39.488406,4.717558,9.117624


i included the feed(raw material input) as well and across the stages as the concentrates enter rough purification they gain a lot concentration from the initial feed, after this they enter primary purification where gold and lead gains more concentration while silver loses it, during our second purification gold and lead lose purification while silver gains concentration, in our final purification gold sees a large concentrate increas while lead gtes a smaller increase and finally silver loses a good portion of its concentration

In [83]:
# Prepare DataFrame for plotly
train_sizes = data_train['primary_cleaner.input.feed_size'].dropna().to_frame(name='feed_size')
train_sizes['set'] = 'Train'

test_sizes = data_test['primary_cleaner.input.feed_size'].dropna().to_frame(name='feed_size')
test_sizes['set'] = 'Test'

combined_sizes = pd.concat([train_sizes, test_sizes], ignore_index=True)

# Plot using Plotly Express
fig = px.histogram(
    combined_sizes, 
    x='feed_size', 
    color='set', 
    marginal='box',
    nbins=50,
    opacity=0.6,
    barmode='overlay',
    title="Feed Particle Size Distribution: Train vs Test"
)
fig.update_layout(xaxis_title='Feed Size', yaxis_title='Count')
fig.show()

i tested the particle distributions and the distributions looks the same just the volume seems to differ which is fine.

the above box chart also shows us some glaring values in our distributions in the from of outliers that id like to drop so ill do so now

In [84]:
# Extract feed sizes
train_sizes = data_train['primary_cleaner.input.feed_size'].dropna().to_frame(name='feed_size')
train_sizes['set'] = 'Train'

test_sizes = data_test['primary_cleaner.input.feed_size'].dropna().to_frame(name='feed_size')
test_sizes['set'] = 'Test'

# Define IQR-based outlier removal function
def drop_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers
train_cleaned = drop_outliers_iqr(train_sizes, 'feed_size')
test_cleaned = drop_outliers_iqr(test_sizes, 'feed_size')

# Combine cleaned data
combined_cleaned = pd.concat([train_cleaned, test_cleaned], ignore_index=True)

# Plot with Plotly Express
fig = px.histogram(
    combined_cleaned,
    x='feed_size',
    color='set',
    marginal='box',
    nbins=50,
    opacity=0.6,
    barmode='overlay',
    title="Feed Particle Size Distribution (Outliers Removed): Train vs Test"
)
fig.update_layout(xaxis_title='Feed Size', yaxis_title='Count')
fig.show()

this is a better example and these are the abnormal values ive decided to drop

The above function should allow use to find the final sMAPE after we decide our best model

In [86]:
# Define target columns
target_columns = ['rougher.output.recovery', 'final.output.recovery']

# Drop rows with missing targets
data_train = data_train.dropna(subset=target_columns)

# Extract features (shared columns between train and test excluding 'date')
features = list(set(data_train.columns) & set(data_test.columns))
features = [f for f in features if f != 'date']

# Define X and target variables
X = data_train[features].fillna(data_train[features].median())
y_rougher = data_train['rougher.output.recovery']
y_final = data_train['final.output.recovery']

# Define sMAPE scoring function
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(np.abs(y_pred - y_true) / denominator) * 100

# Wrap in scorer for cross_val_score (negated to allow higher=better logic)
smape_scorer = make_scorer(smape, greater_is_better=False)

# Models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=25, random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42)
}

# Perform cross-validation for each model
cv = KFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for name, model in models.items():
    smape_rougher = cross_val_score(model, X, y_rougher, scoring=smape_scorer, cv=cv)
    smape_final = cross_val_score(model, X, y_final, scoring=smape_scorer, cv=cv)
    final_smape_score = -0.5 * (smape_rougher.mean() + smape_final.mean())
    results[name] = final_smape_score

results


{'Linear Regression': np.float64(10.067882360654274),
 'Random Forest': np.float64(7.4305420873871935),
 'Decision Tree': np.float64(8.99668740059377)}

after some back and forth i decided to clump smape, 3 model trains, and the cross validation in one action and the above is the result sof that we see that linear regression has a smape of 10.06%, we also see that the randomforest model shows a 7.43% , and finally they decicion tree model shows us 8.99% these finding conclude that randomforest is our best model and we will proceed with this one.

## Conclusion

To predict gold recovery more accurately, we trained and evaluated three regression models — Linear Regression, Random Forest Regressor, and Decision Tree Regressor — using cross-validation and the final sMAPE metric. This metric reflects the model’s accuracy on two critical targets: rougher.output.recovery and final.output.recovery.

The results were as follows:

Random Forest achieved the lowest final sMAPE (~7.43%), indicating the best generalization and accuracy across both targets.

Linear Regression performed reasonably well with a final sMAPE of ~10.07%.

Decision Tree Regressor  offered faster training and interpretability.

Based on these results, Random Forest Regressor was selected as the final model due to its superior balance of accuracy. It will be used to make predictions on the test sample for final evaluation.